In [1]:
import numpy as np
import pandas as pd

import decouple
import sqlalchemy

In [2]:
#Using enviroment variables for security
user = decouple.config("db_user_mysql")
host = decouple.config('db_host_mysql')
password = decouple.config('db_password_mysql')
database = decouple.config("db_database_mysql")
connection = sqlalchemy.create_engine("mysql+mysqldb://"+user+":"+password+"@"+host+"/"+database)
# sudo apt-get install libmysqlclient-dev

In [6]:
df = pd.read_sql_table('text04', con = connection)
df.head(5).T

,0,1,2,3,4
counter,1,2,3,4,5
paciente,356.0,317.0,188.0,53.0,175.0
nometexto,Pedido exame,Pedido exame,Atestado,Pedido exame,Atestado
texto,<h2>Requisição de exames</h2>\n<br/>\n<p>&nbsp...,<h2>Requisição de exames</h2>\n<br/>\n<p>&nbsp...,<h2>Atestado por Acidente de Trabalho</h2>\n<b...,<h2>Requisição de exames</h2>\n<br/>\n<p>&nbsp...,<h2>Atestado de Sanidade Física e Mental</h2>\...
datatexto,2016-07-03 09:41:03,2016-07-06 18:53:30,2016-07-09 16:31:48,2016-07-12 20:47:14,2016-07-19 06:10:31
timestamp,2021-07-01 18:06:19,2021-07-01 18:06:19,2021-07-01 18:06:19,2021-07-01 18:06:19,2021-07-01 18:06:19
cloud,1,0,0,1,1
update001,2021-05-18 12:51:57,2021-02-14 22:10:44,2021-05-24 20:12:22,2021-04-15 05:15:22,2020-11-24 20:06:14
ch3,1,1,1,1,1
sysuser,5,5,6,2,7


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   counter    350 non-null    int64         
 1   paciente   349 non-null    float64       
 2   nometexto  350 non-null    object        
 3   texto      350 non-null    object        
 4   datatexto  350 non-null    datetime64[ns]
 5   timestamp  350 non-null    datetime64[ns]
 6   cloud      350 non-null    int64         
 7   update001  350 non-null    datetime64[ns]
 8   ch3        350 non-null    int64         
 9   sysuser    350 non-null    int64         
dtypes: datetime64[ns](3), float64(1), int64(4), object(2)
memory usage: 27.5+ KB


In [5]:
# Colocando o nome das colunas do arquivo de acordo a documentação da iClinic
df = df.rename(
    columns = {
        "paciente":"patient_id",
        "sysuser": "physician_id",
        "nometexto": "eventprocedure_pack", #tratar depois
        "texto": "description",
        "datatexto": "date",
        "timestamp": "start_time",
        "update001": "end_time",
    }
)

In [6]:
df['status'] = "cp"
df['patient_mobile_phone'] = np.nan
df['arrival_time'] = np.nan
df['all_day'] = np.nan
df['cancel_reason'] = np.nan
df['event_blocked_scheduling'] = np.nan

#tratar depois
# df['patient_name'] = 
# df['patient_home_phone'] =
# df['patient_email'] =
# df['healthinsurance_name'] =

In [ ]:
df['eventprocedure_pack'] = 'json::[\\n\\t{\\n\\t\\t"name":"'+df['eventprocedure_pack']+'"\\n\\t}\\n]'

In [7]:
# Tratamento 'birth_date'
# Não tem valores nulos
df['birth_date'] = pd.to_datetime(df['birth_date'],format='%Y%m%d').dt.date

In [8]:
# Tratamento 'gender'
# Não tem valores nulos
df['gender'] = ['m' if x == 'M' else 'f' if x == 'F' else np.nan for x in df['gender']]
# print(df['gender'].value_counts())

In [9]:
# Tratamento 'mobile_phone'
# Não tem valores nulos
rows_t = df['mobile_phone'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int).astype('str')
# print(rows_t)
# print(rows_t.str.len().value_counts())
# como só tenho numeros de telefone a partir de 10 digitos, então o tipo de telefone mais adequado é o 'home_phone' por acreditar que é o padrão dos pacientes
df['home_phone'] = '(' + rows_t.str[-10:-8] + ')' + rows_t.str[-8:-4] + '-' + rows_t.str[-4:]
df['mobile_phone'] = np.nan

In [10]:
# Tratamento 'zip_code'
# Não tem valores nulos
rows_with_dashes = df['zip_code'].str.contains('-')
# print(rows_with_dashes.value_counts())
# tem valores contendo '-'
df['zip_code'] = [df['zip_code'][i] if x == True else df['zip_code'][i][:5]+'-'+df['zip_code'][i][5:] if x == False else np.nan for i, x in enumerate(rows_with_dashes)]

In [11]:
# Tratamento 'address'
# Não tem valores nulos
# Separando as informações e distribuindo de acordo ao seu tipo
# print(df['address'])
row_with_adress = df['address'].str.split(',')
# print(row_with_adress.str.len().value_counts())
# existem linhas contrando 1 e dois valores apenas
df['number'] = [np.nan if len(x) == 1 else x[1].strip() for x in row_with_adress]
df['address'] = [x[0].strip() for x in row_with_adress]

In [12]:
# Tratamento 'ethnicity'
# Não tem valores nulos
# print(df['ethnicity'].value_counts())
df['ethnicity'] = df['ethnicity'].replace({'Branco': 'wh', 'Amarelo': 'ye', 'Pardo': 'br','Preto': 'bl'})

In [13]:
# Tratamento 'marital_status'
# Não tem valores nulos
# print(df['marital_status'].value_counts())
df['marital_status'] = df['marital_status'].replace({'S': 'si', 'C': 'ma', 'O': 'st', 'V': 'wi','D': 'se'})
# acima foi considerada a sigla "O" como união estável

In [14]:
# Tratamento 'education'
# Não tem valores nulos
# print(df['education'].value_counts())
df['education'] = df['education'].replace({'Ensino Médio': 's', 'Superior': 'h', 'Pós-graduação': 'p', 'Especialização': 'p'})
# acima foi considerada a sigla "Pós-graduação" e "Especialização" como 'p', já que não tem nada para específicar se a pós gradução é lato sensu ou se não foi informado ter sido mestrado

In [15]:
# Tratamento 'indication'
# Todos os valores são nulos
df['indication']= 'ot'

In [16]:
# Tratamento 'active'
# Não tem valores nulos
# print(df['active'].value_counts())
df['active'] = df['active'].replace({0: 1, 1: 0})
# df['active'] = df['active'].astype(np.dtype(bool))

In [17]:
# Tratamento 'picture_filename'
df['picture_filename'] = df['picture_filename'].replace({'NONE': np.nan})
df['picture_filename'] = df['picture_filename'].fillna(np.nan)
# print(df['picture_filename'])

In [18]:
# abaixo estamos removendo os valores que não representam um endereço na pasta 'extra/'
rows_begining_with_extra = df['picture_filename'].str.contains(r'^extra')
df['picture_filename'] = [df['picture_filename'][i][6:] if x == True else 'None' for i, x in enumerate(rows_begining_with_extra)]

In [20]:
df['picture_filename'] = ['picture/'+df['picture_filename'][i] if x == True else 'None' for i, x in enumerate(rows_begining_with_extra)]
# os arquivos de foto do paciente devem ser armazenados junto com os arquivos CSV, na subpasta “picture”.

In [21]:
# Tratamento 'healthinsurance_pack'
df['healthinsurance_pack'] = df['healthinsurance_pack'].replace({np.nan: 0})
df['healthinsurance_pack'] = df['healthinsurance_pack'].astype(int)
df['healthinsurance_pack'] = [x if x <= 35 else 0 for x in df['healthinsurance_pack'] ]
# print(df['healthinsurance_pack'].values)

In [22]:
df_planos = pd.read_csv('desafio-base2-output/planos.csv',index_col = "plan")
df_planos.head(7).T

plan,1,2,3,4,5,6,7
code,9118087,518103,1377066,3346654,429694,1561209,2031645
name,Unimed,Bradesco,Amil,NotreDame,Porto Seguro,SulAmérica,Allianz
account,9823,6813,5061,4479,4058,9954,2695
billsenddayinternal,1,20,21,25,1,13,12
returndaysexternal,20,21,25,29,29,21,22


In [23]:
# Neste caso, como o 'healthinsurance_pack' inicia com 1 e está ordenada é fácil de encontrar seu nome correspondente
index = df['healthinsurance_pack'].values
# print(index)
df['healthinsurance_pack_name'] = [ df_planos['name'][i] if i != 0 else np.nan for i in index ]
df['healthinsurance_pack_code'] = [ df_planos['code'][i].astype(int).astype(str) if i != 0 else np.nan for i in index ]

# print(df['healthinsurance_pack_name'])

In [24]:
df['healthinsurance_pack'] = 'json::[\\n\\t{\\n\\t\\t"name":"'+df['healthinsurance_pack_name']+'",\\n\\t\\t"code":"'+df['healthinsurance_pack_code']+'"\\n\\t}\\n]'

In [25]:
df = df.loc[:, ["patient_id","patient_name","physician_id","date","status","patient_home_phone","patient_mobile_phone","arrival_time","start_time","end_time","description","all_day","cancel_reason","patient_email","event_blocked_scheduling","healthinsurance_name","eventprocedure_pack"]]

In [26]:
df.to_csv('desafio-base2-output/event_scheduling.csv',index=False, encoding='utf-8')